In [ ]:
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
import sln_letter_fit
from sln_letter_fit import FitParams
from sln_stroke_fit import OptimizationLoggingParams

%load_ext autoreload
%autoreload 1
%aimport sln_letter_fit

# Conventions:

* Trajectory - full set of points from pen down to pen up
* Stroke - subset of trajectory segmented by velocity curve
* Segment - either a stroke or trajectory, depending on context

# Loading and Plotting Utilities

In [ ]:
DT = 1./120
def load_letter(letter='D'):
    with np.load('../all_letters.npz', allow_pickle=True) as data:
        return data[letter]
def split_trajectory(trajectory, debug=False):
    v = np.diff(trajectory[:, 1:], axis=0)
    speed2 = np.sum(np.square(v), axis=1)
    speed2_smooth = scipy.signal.savgol_filter(speed2, 5, 1)
    # find local minima in speeds
    splits, _ = scipy.signal.find_peaks(-speed2_smooth, width=0.02 * 120)
    splits = list(filter(lambda i: i > 0.05 * 120, splits))

    if debug:
        plt.subplot(121)
        # plt.plot(speed2, 'k-')
        plt.plot(speed2_smooth, '-', linewidth=1)
        plt.plot(splits, speed2[splits], 'r*')
        plt.subplot(122)
        plt.plot(trajectory[:, 1], trajectory[:, 2])
        plt.plot(trajectory[splits, 1], trajectory[splits, 2], 'r*')

    splits = [0, *splits, trajectory.shape[0] - 1]
    return [trajectory[begin:end, :] for begin, end in zip(splits[:-1], splits[1:])]
def trajectory2strokes(trajectory, debug=False):
    t = np.arange(0, trajectory.shape[0] * DT, DT).reshape(-1, 1)
    return split_trajectory(np.hstack((t, trajectory)), debug=debug)
def load_segments(letter='D', index=1, debug=False):
    """ If index is None, load all trajectories.
        If index is an int, load all the strokes in that one trajectory."""
    letter_data = load_letter(letter)
    if index is None:  # return all strokes
        return [trajectory2strokes(letter_data[i], debug=debug) for i in range(1, len(letter_data), 2)]
    return trajectory2strokes(letter_data[index], debug=debug)
# Debug:
# load_segments('A', 1, debug=True);
for letter in 'ABCD':
    plt.figure(figsize=(10, 5))
    load_segments(letter, index=None, debug=True);
    plt.show()

In [ ]:
def plot_trajectory(ax, strokes, sol, iteration_number=None):
    txy_gt = np.vstack(strokes)
    txy_params = sol['txy_from_params']
    ax.plot(txy_gt[:, 1], txy_gt[:, 2], 'k.', label='Mocap data')
    colors = 'rgb'
    for i, (begin, end) in enumerate(sol['stroke_indices'].values()):
        color = colors[i % 3]
        ax.plot(txy_params[begin:end, 1], txy_params[begin:end, 2], color + '-', \
                label='Optimized SLN curve', linewidth=1)
        ax.plot(sol['txy'][begin:end, 1], sol['txy'][begin:end, 2], color + '.')
    def expand_range(vec, padding=0.1):
        l, h = np.min(vec), np.max(vec)
        return l - (h - l) * padding, h + (h - l) * padding
    ax.axis('equal')
    ax.set_xlim(*expand_range(txy_gt[:, 1]))
    ax.set_ylim(*expand_range(txy_gt[:, 2]))
    # ax.set_aspect('equal', adjustable='box')
    if iteration_number is not None:
        ax.text(0.05, 0.9, 'Iter {:}'.format(iteration_number), transform=ax.transAxes)
    # Display params on plot:
    text = '\n'.join('params: {:}'.format(np.round(params, 2)) for params in sol['params'])
    text_box = matplotlib.offsetbox.AnchoredText(text,
                                                 frameon=True,
                                                 loc='lower right',
                                                 pad=0.3,
                                                 bbox_to_anchor=(1, 0),
                                                 bbox_transform=ax.transAxes,
                                                 borderpad=0,
                                                 prop=dict(size=9))
    text_box.patch.set_alpha(0.4)
    ax.add_artist(text_box)
    ax.legend(loc='upper right')
def plot_letter(ax, trajectories, sols):
    for strokes, sol in zip(trajectories, sols):
        plot_trajectory(ax, strokes, sol)
    ax.get_legend().remove()
animation_oversample = 1
save_animation_fname = None
def animate_trajectories(ax, trajectories, sols_and_histories):
    def update(i):
        ax.cla()
        for strokes, (_, history) in zip(trajectories, sols_and_histories):
            plot_trajectory(ax, strokes, history[min(i, len(history) - 1)], iteration_number=i)
        ax.get_legend().remove()
        ax.axis('equal')

    max_iterations = max(len(history) for _, history in sols_and_histories)
    with tqdm.notebook.trange(0, max_iterations, animation_oversample) as progress_bar:
        progress_bar.set_description('Creating Animation')
        ani = matplotlib.animation.FuncAnimation(fig, update, frames=progress_bar)
        if save_animation_fname is not None:
            print("Saving animation...")
            ani.save(save_animation_fname, writer=matplotlib.animation.FFMpegWriter(fps=60))
            print("Saved.")
            progress_bar.reset()
        return HTML(ani.to_jshtml())

In [ ]:
# Convenience functions to both fit and plot at the same time
def fit_and_plot_trajectory(ax, strokes, max_iters: int, log_history: bool, pbar_description: str):
    sol, history, fitter, _ = sln_letter_fit.fit_letter(
        strokes,
        fit_params=FitParams(max_iters=max_iters, initialization_strategy_params=' :D '),
        optimization_logging_params=OptimizationLoggingParams(
            log_optimization_values=log_history,
            progress_bar_class=tqdm.tqdm_notebook,
            progress_bar_description=pbar_description))
    plot_trajectory(ax, strokes, sol)
    return sol, history

def fit_and_plot_trajectories(ax,
                        letter,
                        num_strokes=None,
                        trajectory_indices=(0,),
                        max_iters=100,
                        log_history=False,
                        animate=False):
    """
    Returns:
        sols_and_histories:
            For each trajectory,
                Returns 2-tuple of sol, history
    """
    all_trajectories = load_segments(letter, index=None)
    if trajectory_indices is not None:
        all_trajectories = [all_trajectories[i] for i in trajectory_indices]
    all_returns = []
    for traji, strokes in enumerate(all_trajectories):
        if num_strokes is None:
            num_strokes = len(strokes)
        sol, history = fit_and_plot_trajectory(
            ax,
            strokes[:num_strokes],
            max_iters,
            log_history or animate,
            pbar_description='Fitting Letter {:}, traj {:}'.format(letter, traji))
        all_returns.append((sol, history))
    if animate:
        return all_returns, animate_trajectories(ax, all_trajectories, all_returns)
    return all_returns

# Fit and Plot the letter 'D'

In [ ]:
# Don't use convenience function, so that we can see more clearly what's going on
strokes = load_segments('D', 1)
sol, history, fitter, _ = sln_letter_fit.fit_letter(
    strokes,
    fit_params=FitParams(max_iters=50, initialization_strategy_params=' :D '),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=False,
                                                          progress_bar_class=tqdm.tqdm_notebook))
fig, ax = plt.subplots()
plot_trajectory(ax, strokes, sol, iteration_number=len(history))
plt.show()

In [ ]:
fig, ax = plt.subplots()
_, anim = fit_and_plot_trajectories(ax, 'D', num_strokes=None, max_iters=50, animate=True)
plt.close()
anim

# Playing around

In [ ]:
# Plot the hole in the letter "A"
fit_and_plot_trajectories(ax,
                          'A',
                          num_strokes=3,
                          trajectory_indices=[1],
                          max_iters=100,
                          animate=True)[1]

In [ ]:
animation_oversample = 1
save_animation_fname = 'data/optimization_animation_C_allstrokes.mp4'
fig, ax = plt.subplots(figsize=(5, 4))
_, anim = fit_and_plot_trajectories(ax,
                                    'C',
                                    num_strokes=None,
                                    trajectory_indices=(0,),
                                    max_iters=250,
                                    log_history=True,
                                    animate=True)
save_animation_fname = None
plt.close()
anim

# Fit and plot lots of letters

In [ ]:
def fit_and_plot_letters(letters, **kwargs):
    fig, axes = plt.subplots(1, len(letters), figsize=(25, 5))
    for ax, letter in zip(axes, letters):
        fit_and_plot_trajectories(ax, letter, **kwargs) # animate doesn't work
        ax.get_legend().remove()
        ax.axis('equal')

In [ ]:
fit_and_plot_letters('ABCD', num_strokes=None, trajectory_indices=None, max_iters=250)

In [ ]:
# initializations:
fit_and_plot_letters('ABCD', num_strokes=None, trajectory_indices=None, max_iters=1)

In [ ]:
animation_oversample = 10
for letter in 'ABCDE':
    save_animation_fname = 'data/optimization_animation_{:}_allstrokes_fast.mp4'.format(letter)
    fig, ax = plt.subplots(figsize=(5, 4))
    _, anim = fit_and_plot_trajectories(ax,
                                        letter,
                                        num_strokes=None,
                                        # trajectory_indices=(0,),
                                        trajectory_indices=None,
                                        max_iters=250,
                                        log_history=True,
                                        animate=True)
    save_animation_fname = None
    plt.close()